In [1]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import  RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
from model_training import ModelTrainer
from data_cleaning import DataCleaner
from feature_extraction import FeatureExtractor

In [3]:
# Ensure mlflow is available in the current kernel.
import importlib, sys, subprocess
try:
    import mlflow
    print("mlflow is already installed.")
except ModuleNotFoundError:
    print("mlflow not found, installing via pip into the current environment...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'mlflow'])
    import mlflow
    print("mlflow installed and imported.")
# Optional: check versions
try:
    print('mlflow version:', mlflow.__version__)
except Exception:
    pass

mlflow is already installed.
mlflow version: 3.5.1


In [4]:
from mlflow.models import infer_signature

In [5]:
original_news_df = pd.read_csv('../data/raw/online_news_original.csv')
original_news_df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,http://mashable.com/2014/12/27/samsung-app-aut...,8.0,11.0,346.0,0.529052,1.0,0.684783,9.0,7.0,1.0,...,0.100000,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800
39640,http://mashable.com/2014/12/27/seth-rogen-jame...,8.0,12.0,328.0,0.696296,1.0,0.885057,9.0,7.0,3.0,...,0.136364,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900
39641,http://mashable.com/2014/12/27/son-pays-off-mo...,8.0,10.0,442.0,0.516355,1.0,0.644128,24.0,1.0,12.0,...,0.136364,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900
39642,http://mashable.com/2014/12/27/ukraine-blasts/,8.0,6.0,682.0,0.539493,1.0,0.692661,10.0,1.0,1.0,...,0.062500,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100


In [7]:
y = original_news_df['shares'].copy()
X = original_news_df.drop(columns=['shares'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
params={
    "n_estimators": 2,
    "max_depth": 10,
    "random_state": 10
}

In [9]:
# mlflow ui --backend-store-uri mlruns
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("MLFlow Pipeline Test")


2025/11/04 04:15:33 INFO mlflow.tracking.fluent: Experiment with name 'MLFlow Pipeline Test' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/746760287326177225', creation_time=1762251334015, experiment_id='746760287326177225', last_update_time=1762251334015, lifecycle_stage='active', name='MLFlow Pipeline Test', tags={}>

In [10]:
with mlflow.start_run():
    pipeline = Pipeline([
        ('cleaning', DataCleaner()),
        ('features', FeatureExtractor()),
        ('trainer', ModelTrainer(
            model_class=RandomForestClassifier,
            model_params=params    ))
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy) 

    signature = infer_signature(X_train, pipeline.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=pipeline,
        name="Pipeline Test",
        signature=signature,
        input_example=X_train,
        registered_model_name="pipeline-test"
    )

    mlflow.set_logged_model_tags(
        model_info.model_id,
        {
            "Training Info": "Basic RF model for news data",
        
        }
    )

c:\Users\jptl2\Desktop\MLOpsEq15\MLOpsEq15\.venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\jptl2\Desktop\MLOpsEq15\MLOpsEq15\.venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\jptl2\Desktop\MLOpsEq15\MLOpsEq15\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jptl2\Desktop\MLOpsEq15\MLOpsEq15\.venv\Lib\site-packages\sklearn

🏃 View run fortunate-fish-193 at: http://127.0.0.1:5000/#/experiments/746760287326177225/runs/fc0e33e7c21647fea27aa533b4e92fa9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/746760287326177225
